In [34]:
!pip install xgboost
import pandas as pd
import xgboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [35]:


data_path = "./Files/After_Feature_Engineering/Split/"


# The Project Consist of 3 parts : 


# Load the data from the 'data_processed.csv' file 

#1 PART C
X_train_C_part1_FE = pd.read_csv(data_path+'X_train_C_part1_FE.csv', low_memory=False)
X_test_C_part1_FE = pd.read_csv(data_path+'X_test_C_part1_FE.csv',low_memory=False)

Y_train_C_part1_FE = pd.read_csv(data_path+'Y_train_C_part1_FE.csv',low_memory=False)
Y_test_C_part1_FE = pd.read_csv(data_path+'Y_test_C_part1_FE.csv',low_memory=False)




X_train_C_part2_FE = pd.read_csv(data_path+'X_train_C_part2_FE.csv',low_memory=False)
X_test_C_part2_FE = pd.read_csv(data_path+'X_test_C_part2_FE.csv',low_memory=False)

Y_train_C_part2_FE = pd.read_csv(data_path+'Y_train_C_part2_FE.csv',low_memory=False)
Y_test_C_part2_FE = pd.read_csv(data_path+"Y_test_C_part2_FE.csv",low_memory=False)





X_train_D_FE = pd.read_csv(data_path+"X_train_D_FE.csv",low_memory=False)
X_test_D_FE = pd.read_csv(data_path+"X_test_D_FE.csv",low_memory=False)

Y_train_D_FE = pd.read_csv(data_path +"Y_train_D_FE.csv",low_memory=False)
Y_test_D_FE = pd.read_csv(data_path +"Y_test_D_FE.csv",low_memory=False)



In [36]:
#RUN in 10 Min 

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier



# Hyperparameters to try
n_estimators_values = np.arange(10, 30, 10)
n_neighbors = np.arange(1,10,1)

# Define a list of estimators with their respective hyperparameters
estimators = [

    


    (BaggingRegressor(), {'n_estimators': n_estimators_values}), #0.37110787901763215
    (GradientBoostingRegressor(),{'n_estimators': n_estimators_values}),# 1.6587493376697904 4 Min 44
    (KNeighborsRegressor(),{'n_neighbors':n_neighbors}), #1.8000806798211735

  
    (RandomForestRegressor(), {'n_estimators': n_estimators_values})
]#

# Different k values for k-fold cross-validation

cv_values = [5]

# Store the best model, its parameters, and performance
best_model = None
best_cv = None
best_mae = float('inf')
all_results = []

#Zip data for reusable code
regression_columns_part1 = ["Total number studs","HoldDown Model / ATS"]
Regression_column_part2 = ["Tx(s)","Ty(s)"]
regression_columns_D = ['Ωx', 'Ωy', 'µx', 'µy', 'CMR', 'SSF', 'ACMR', 'IO-ln θ', 'IO-β','LS-ln θ', 'LS-β', 'CP-ln θ', 'CP-β']



regression_collumns_grouped = [regression_columns_part1,Regression_column_part2,regression_columns_D]


data = [(X_train_C_part1_FE,X_test_C_part1_FE,Y_train_C_part1_FE,Y_test_C_part1_FE),(X_train_C_part2_FE,X_test_C_part2_FE,Y_train_C_part2_FE,Y_test_C_part2_FE),(X_train_D_FE,X_test_D_FE,Y_train_D_FE,Y_test_D_FE)]




for regression_collumns,data in zip(regression_collumns_grouped,data):

    X_train, X_test, Y_train, Y_test  = data
    for regression_col in regression_collumns:

        Y_train_reg = Y_train[regression_col]
        Y_test_reg = Y_test[regression_col]

        for cv in cv_values:
            results = []

            # Iterate through the estimators and their hyperparameters
            for estimator, param_grid in estimators:


                grid_search = GridSearchCV(estimator, param_grid, cv=cv, scoring='neg_mean_squared_error')
                grid_search.fit(X_train, Y_train_reg)
                

                Y_pred = grid_search.predict(X_test)


                mae = mean_absolute_error(Y_test_reg, Y_pred)
                
                results.append({
                    'CV': cv,
                    'Estimator': estimator.__class__.__name__,
                    'Best Parameters': grid_search.best_params_,
                    'Mean Absolute Error': mae,
                    'Column':regression_col

                })
                
                if mae < best_mae:
                    best_mae = mae
                    best_cv = cv
                    best_model = grid_search.best_estimator_
                    best_params = grid_search.best_params_
                    

            all_results.extend(results)

# Convert all results to a DataFrame
all_results_df = pd.DataFrame(all_results)



In [37]:
all_results_df = pd.DataFrame(all_results)
idx = all_results_df.groupby("Column")['Mean Absolute Error'].idxmin()



result = all_results_df.loc[idx]
result.reset_index(drop=True, inplace=True)

result

,CV,Estimator,Best Parameters,Mean Absolute Error,Column
0,5,RandomForestRegressor,{'n_estimators': 20},0.329638,ACMR
1,5,RandomForestRegressor,{'n_estimators': 20},0.228437,CMR
2,5,RandomForestRegressor,{'n_estimators': 20},0.057420,CP-ln θ
3,5,RandomForestRegressor,{'n_estimators': 10},0.023143,CP-β
4,5,BaggingRegressor,{'n_estimators': 20},0.088015,HoldDown Model / ATS
5,5,RandomForestRegressor,{'n_estimators': 20},0.057140,IO-ln θ
6,5,BaggingRegressor,{'n_estimators': 20},0.014485,IO-β
7,5,RandomForestRegressor,{'n_estimators': 20},0.055112,LS-ln θ
8,5,RandomForestRegressor,{'n_estimators': 20},0.019910,LS-β
9,5,RandomForestRegressor,{'n_estimators': 10},0.008325,SSF


In [38]:
""" RUN WITH ALL REGRESSIONS

   CV                  Estimator                  Best Parameters  \
0   5           BaggingRegressor  {'estimator__n_estimators': 75}   
1   5  GradientBoostingRegressor  {'estimator__n_estimators': 95}   
2   5      RandomForestRegressor  {'estimator__n_estimators': 80}   

   Mean Squared Error  
0            0.333335  
1            1.680206  
2            0.334892  

Overall Best Model Results:
Best CV: 5
Best Mean Squared Error: 0.33333456516122734
Best Hyperparameters: {'estimator__n_estimators': 75}
Best Model: MultiOutputRegressor(estimator=BaggingRegressor(n_estimators=75))





"""


" RUN WITH ALL REGRESSIONS\n\n   CV                  Estimator                  Best Parameters  0   5           BaggingRegressor  {'estimator__n_estimators': 75}   \n1   5  GradientBoostingRegressor  {'estimator__n_estimators': 95}   \n2   5      RandomForestRegressor  {'estimator__n_estimators': 80}   \n\n   Mean Squared Error  \n0            0.333335  \n1            1.680206  \n2            0.334892  \n\nOverall Best Model Results:\nBest CV: 5\nBest Mean Squared Error: 0.33333456516122734\nBest Hyperparameters: {'estimator__n_estimators': 75}\nBest Model: MultiOutputRegressor(estimator=BaggingRegressor(n_estimators=75))\n\n\n\n\n\n"

In [39]:
##Run in 12 m

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


# Hyperparameters to try
n_estimators_values = np.arange(10, 31, 10)
n_neighbors = np.arange(1,10,1)

# Define a list of estimators with their respective hyperparameters
estimators = [
    (LogisticRegression(max_iter=1000),{}),
    (RandomForestClassifier(),{'estimator__n_estimators': n_estimators_values}),
    (DecisionTreeClassifier(),{}),
    (KNeighborsClassifier(),{'estimator__n_neighbors':n_neighbors})

]#

# Different k values for k-fold cross-validation

cv_values = [5]

# Store the best model, its parameters, and performance
best_model = None
best_cv = None
best_f1=0
best_accuracy=0
all_results = []



categorical_columns = ["Nail spacing [cm]","Number sheathing panels","Number end studs"]


for cat_coll in categorical_columns:
    Y_train_cat = pd.get_dummies(Y_train_C_part1_FE[cat_coll],columns=cat_coll)
    Y_test_cat = pd.get_dummies(Y_test_C_part1_FE[cat_coll],columns=cat_coll)


# Iterate over different values of k for cross-validation
    for cv in cv_values:
        results = []

        # Iterate through the estimators and their hyperparameters
        for estimator, param_grid in estimators:
            regressor = MultiOutputClassifier(estimator)
            grid_search = GridSearchCV(regressor, param_grid, cv=cv)
            grid_search.fit(X_train_C_part1_FE, Y_train_cat)
            

            Y_pred = grid_search.predict(X_test_C_part1_FE)
    


            accuracy = accuracy_score(Y_test_cat, Y_pred)
            f1 = f1_score(Y_test_cat, Y_pred,average="micro")
            results.append({
                'CV': cv,
                'Estimator': estimator.__class__.__name__,
                'Best Parameters': grid_search.best_params_,
                'best_score ': grid_search.best_score_,
                'Best accuracy':accuracy,
                'Column':cat_coll

            })
            
            if accuracy > best_accuracy:
                best_cv = cv
                best_model = grid_search.best_estimator_
                best_params = grid_search.best_params_
                best_accuracy = accuracy


        all_results.extend(results)

# Convert all results to a DataFrame
all_results_df = pd.DataFrame(all_results)


# Display the overall best model and its performance
print("\nOverall Best Model Results:")
print(f"Best CV: {best_cv}")
print(f"Best Hyperparameters: {best_params}")
print(f"Best Model: {best_model}")
print(f"Best accuracy: {best_accuracy}")



Overall Best Model Results:
Best CV: 5
Best Hyperparameters: {}
Best Model: MultiOutputClassifier(estimator=DecisionTreeClassifier())
Best accuracy: 0.9845820284268851


In [40]:
all_results_df = pd.DataFrame(all_results)
idx = all_results_df.groupby("Column")['Best accuracy'].idxmax()



result = all_results_df.loc[idx]
result.reset_index(drop=True, inplace=True)

result



,CV,Estimator,Best Parameters,best_score,Best accuracy,Column
0,5,DecisionTreeClassifier,{},0.865890,0.889344,Nail spacing [cm]
1,5,DecisionTreeClassifier,{},0.964043,0.973982,Number end studs
2,5,DecisionTreeClassifier,{},0.977233,0.984582,Number sheathing panels


In [41]:
X_train_D_FE_ATS = X_train_D_FE[X_train_D_FE["connection_system_ATS"]==1]
Y_train_D_FE_ATS = Y_train_D_FE.loc[X_train_D_FE_ATS.index]

X_test_D_FE_ATS = X_test_D_FE[X_test_D_FE["connection_system_ATS"]==1]
Y_test_D_FE_ATS = Y_test_D_FE.loc[X_test_D_FE_ATS.index]

X_train_D_FE_HD = X_train_D_FE[X_train_D_FE["connection_system_HD"]==1]
Y_train_D_FE_HD= Y_train_D_FE.loc[X_train_D_FE_HD.index]

X_test_D_FE_HD = X_test_D_FE[X_test_D_FE["connection_system_HD"]==1]
Y_test_D_FE_HD = Y_test_D_FE.loc[X_test_D_FE_HD.index]




#RUN in 10 Min 

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier



# Hyperparameters to try
n_estimators_values = np.arange(10, 30, 10)
n_neighbors = np.arange(1,10,1)

# Define a list of estimators with their respective hyperparameters
estimators = [

    


    (BaggingRegressor(), {'n_estimators': n_estimators_values}), #0.37110787901763215
    (GradientBoostingRegressor(),{'n_estimators': n_estimators_values}),# 1.6587493376697904 4 Min 44
    (KNeighborsRegressor(),{'n_neighbors':n_neighbors}), #1.8000806798211735

  
    (RandomForestRegressor(), {'n_estimators': n_estimators_values})
]#

# Different k values for k-fold cross-validation

cv_values = [5]

# Store the best model, its parameters, and performance
best_model = None
best_cv = None
best_mae = float('inf')
all_results = []

#Zip data for reusable code

regression_columns_D = ['Ωx', 'Ωy', 'µx', 'µy', 'CMR', 'SSF', 'ACMR', 'IO-ln θ', 'IO-β','LS-ln θ', 'LS-β', 'CP-ln θ', 'CP-β']



regression_collumns_grouped = [regression_columns_D,regression_columns_D]


data = [(X_train_D_FE_ATS,X_test_D_FE_ATS,Y_train_D_FE_ATS,Y_test_D_FE_ATS,"ATS"),(X_train_D_FE_HD,X_test_D_FE_HD,Y_train_D_FE_HD,Y_test_D_FE_HD,"HD")]




for regression_collumns,data in zip(regression_collumns_grouped,data):

    X_train, X_test, Y_train, Y_test,Model_name  = data
    for regression_col in regression_collumns:

        Y_train_reg = Y_train[regression_col]
        Y_test_reg = Y_test[regression_col]

        for cv in cv_values:
            results = []

            # Iterate through the estimators and their hyperparameters
            for estimator, param_grid in estimators:


                grid_search = GridSearchCV(estimator, param_grid, cv=cv, scoring='neg_mean_squared_error')
                grid_search.fit(X_train, Y_train_reg)
                

                Y_pred = grid_search.predict(X_test)


                mae = mean_absolute_error(Y_test_reg, Y_pred)
                
                results.append({
                    'CV': cv,
                    'Estimator': estimator.__class__.__name__,
                    'Best Parameters': grid_search.best_params_,
                    'Mean Absolute Error': mae,
                    'Column':regression_col,
                    'Model_name':Model_name

                })
                
                if mae < best_mae:
                    best_mae = mae
                    best_cv = cv
                    best_model = grid_search.best_estimator_
                    best_params = grid_search.best_params_
                    

            all_results.extend(results)

# Convert all results to a DataFrame
all_results_df = pd.DataFrame(all_results)




In [42]:
all_results_df = pd.DataFrame(all_results)

all_results_df['Combined'] = all_results_df['Column'] + all_results_df['Model_name']

# Find the index of the row with the lowest Mean Absolute Error for each unique value in 'Combined'
idx = all_results_df.groupby('Combined')['Mean Absolute Error'].idxmin()

result = all_results_df.loc[idx]

# Resetting the index if needed
result.reset_index(drop=True, inplace=True)
result = result.drop(columns="Combined")

result

,CV,Estimator,Best Parameters,Mean Absolute Error,Column,Model_name
0,5,BaggingRegressor,{'n_estimators': 10},0.515182,ACMR,ATS
1,5,RandomForestRegressor,{'n_estimators': 20},0.271000,ACMR,HD
2,5,RandomForestRegressor,{'n_estimators': 20},0.320045,CMR,ATS
3,5,RandomForestRegressor,{'n_estimators': 20},0.211328,CMR,HD
4,5,RandomForestRegressor,{'n_estimators': 20},0.050650,CP-ln θ,ATS
5,5,BaggingRegressor,{'n_estimators': 20},0.035936,CP-ln θ,HD
6,5,BaggingRegressor,{'n_estimators': 10},0.013100,CP-β,ATS
7,5,RandomForestRegressor,{'n_estimators': 10},0.024566,CP-β,HD
8,5,BaggingRegressor,{'n_estimators': 20},0.058986,IO-ln θ,ATS
9,5,RandomForestRegressor,{'n_estimators': 20},0.042879,IO-ln θ,HD
